# Identifying Hate Speech in Social Media Comments: PyTorch LSTM

### Import Modules

In [0]:
import os
import re
import sys
import ssl
import json
import nltk
import time
import math
import string
import pickle
import argparse

import torch
import numpy as np
import pandas as pd
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from collections import Counter, OrderedDict
from sklearn.model_selection import train_test_split
from torchtext.data import Field, BucketIterator
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pad_packed_sequence, pack_padded_sequence

In [0]:
import warnings
# warnings.filterwarnings('ignore')

warnings.filterwarnings(action='once')

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# from google.colab import files, auth
# auth.authenticate_user()
uploaded = files.upload()

Saving lstm_results.json to lstm_results.json


In [12]:
import exec_lstm as exl

Using CUDA? True cuda:0


/usr/local/lib/python3.6/dist-packages/nltk/corpus/reader/wordlist.py:28: ResourceWarning: unclosed file <_io.BufferedReader name='/root/nltk_data/corpora/stopwords/english'>
  return concat([self.open(f).read() for f in fileids])


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [14]:
torch.cuda.is_available()

True

In [0]:
# import importlib
# import google
# importlib.reload(google.colab.auth)

In [0]:
# from google.colab import auth
# auth.authenticate_user()

/usr/local/lib/python3.6/dist-packages/google/colab/auth.py:141: ResourceWarning: unclosed <ssl.SSLSocket fd=66, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 56462), raddr=('209.85.145.84', 443)>
  if _check_adc():


In [15]:
!pip install gcsfs

     |████████████████████████████████| 61kB 4.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/9f/0f/b9/5bc5222756d121ccace51ab3084a1c733380908a4e2f939038
Successfully built gcsfs


/usr/local/lib/python3.6/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.6/dist-packages/gcsfs-0.2.2.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):


In [16]:
import gcsfs
train_full = pd.read_csv('gs://no-hate/train.csv')

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [0]:
train_full['label'] = train_full.target.apply(lambda x: 1 if x > 0.5 else 0)

In [0]:
test_ratio = 0.2

## Housekeeping and Variable Storage

In [0]:
torch.cuda.empty_cache()

In [20]:
project_id = 'tribal-monolith-242216'
!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud alpha survey



In [0]:
def send_to_bucket(filename, bucket_name="no-hate", directory=None):
    if directory:
        !gsutil cp /content/{filename} gs://{bucket_name}/{directory}/
    else:
        !gsutil cp /content/{filename} gs://{bucket_name}/
        
    

In [0]:
# send_to_bucket(filename='20pct_imbalanced_model_state.pt')

In [0]:
import io
# import cloudstorage
import gcsfs
from io import BytesIO
import pickle

def pickle_to_gcs(df, filename, bucket_name="no-hate", directory=None):
    """
    Pickle a dataframe to a specified file in a given Google Cloud Storage 
    bucket, optionally within a specified directory.
    
    Arguments:
        df: a pandas dataframe
        filename: filename to write to in a bucket given
        bucket_name: the bucket to write to
        directory: The directory to write to
    
    """
    pickle_buffer = io.BytesIO()
    fs = gcsfs.GCSFileSystem(project=project_id)
    df.to_pickle(filename)
    
    if directory:
        directory = directory + "/"
    else:
        directory = ""
    
    with fs.open(f"{bucket_name}/{directory}{filename}", "wb") as handle:    
        pickle.dump(df, handle)
        
    
def load_pickle_from_gcs(filename, bucket_name="no-hate",  directory=None):
    """     
    Load a pickled dataframe from a specified file in a given Google Cloud 
    Storage bucket, optionally within a specified directory.
    
    Arguments:
        df: a pandas dataframe
        filename: filename to write to in a bucket given
        bucket_name: the bucket to write to
        directory: The directory to write to
    """
    fs = gcsfs.GCSFileSystem(project=project_id) 
    
    if directory:
        directory = directory + "/"
    else:
        directory = ""
        
    with fs.open(f"{bucket_name}/{directory}{filename}", "rb") as handle:
        df = pickle.load(handle)
    return df

In [0]:
# Check Available RAM

# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

In [0]:
# Clear RAM

# !kill -9 -1

## Creating Batch Loader to Enable Larger Dataset Training

Use 20% data partition while building batch loader for LSTM model.

### Create Batch Loader Test Data

In [0]:
train_set, val_set, test_set = exl.get_samples(train_full, proportion=0.2, 
                                                  train_test_ratio=(1-test_ratio))

In [0]:
print(train_sample.shape, val_set.shape, test_set.shape)

In [0]:
train_sample.loc[train_sample.comment_text != ''].shape

In [0]:
train_sample.head()

In [0]:
val_samples = val_set.sample( n=math.ceil(len(train_sample)*test_ratio), random_state=1008 )

In [0]:
val_samples.shape

In [0]:
test_samples = test_set.sample( n=math.ceil(len(train_sample)*test_ratio), random_state=1008 )

### Data Loader Class

In [0]:
from torchtext.data import Field, Dataset, Example
import pandas as pd

class DataFrameDataset(Dataset):
    """
    Class for using pandas DataFrames as a datasource.
    
    H/T for the TorchText dataframe-based-Dataset hack to: 
    https://stackoverflow.com/questions/52602071/dataframe-as-datasource-in-torchtext
    """
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        
        Arguments:
            examples: pandas dataFrame of examples
                fields: dictionary in format {str: Field} Fields to use in this tuple. The
                string is a field name, and the Field is the associated TorchText Field.
            filter_pred: callable or None. Whether to use only exanples for which
                filter_pred(example) is true, or use all examples if None.
                Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]

                
class SeriesExample(Example):
    """
    Class to convert a pandas Series to a TorchText Example
    
    H/T for the TorchText dataframe-based-Dataset hack to: 
    https://stackoverflow.com/questions/52602071/dataframe-as-datasource-in-torchtext
    """

    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()

        for key, field in fields.items():
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex

In [0]:
def tokenizer(text, stop_ws=exl.stops, stemmer=None, str_output=False):
    """
    Tokenize text for deep learning, with handling for comments comprised
    solely of sotp words, which have length zero after tokenization.
    
    Arguments:
        text: text string to be tokenized
        stop_ws: a set of stopwords to remove from text. Default is augmented 
            NLTK stopwords (excluding a handful of preselected keyworrds).
        stemmer: an instance of an NLTK stemmer instance to use for stemming, 
            or None. Default is None.
        str_output: Whether to return a string (versus a list). Default is False.
    """
    t = text.replace("-", " ").split(" ")
    t = [w.strip(string.punctuation) for w in t]

    if stop_ws:
        t = [w.lower() for w in t if w not in stop_ws]

    if stemmer:
        t = [stemmer.stem(w) for w in t]

    if t == []:
        return ['<blank>']
    if str_output:
        return ' '.join(t)
    else:
        return t

### Data Loader Testing

In [0]:
TEXT = Field(tokenize=tokenizer, lower=True, include_lengths = True)
LABEL = Field(sequential=False, unk_token=None, 
              dtype=torch.float)

In [0]:
train_ds = DataFrameDataset(train_sample, fields = {'label' : LABEL, 
                                                    'comment_text': TEXT})
valid_ds = DataFrameDataset(val_samples, fields = {'label' : LABEL, 
                                                   'comment_text': TEXT})
test_ds = DataFrameDataset(test_samples, fields = { 'label' : LABEL, 
                                                   'comment_text' : TEXT})

In [0]:
train_ds.fields

In [0]:
valid_ds.fields

In [0]:
TEXT.build_vocab(train_ds, max_size=25000, 
#                  vectors = "glove.840B.300d",
                 vectors = "glove.6B.200d",
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_ds)

In [0]:
print('len(TEXT.vocab)', len(TEXT.vocab))

In [0]:
print(TEXT.vocab.stoi['hope'])

In [0]:
print(TEXT.vocab.itos[198])

In [0]:
BATCH_SIZE = 1300

In [0]:
train_iter, val_iter, test_iter = BucketIterator.splits(
    (train_ds, valid_ds, test_ds), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.comment_text),
    sort_within_batch = True,
    device = device)

In [0]:
batch = next(iter(train_iter))
print(batch.label)
print(batch.label.size())
print(batch.comment_text)
print(batch.comment_text[0].size())

In [0]:
val_batch = next(iter(val_iter))
print(val_batch.label)
print(val_batch.label.size())
print(val_batch.comment_text)
print(val_batch.comment_text[0].size())

In [0]:
test_batch = next(iter(test_iter))
print(test_batch.label)
print(test_batch.label.size())
print(test_batch.comment_text)
print(test_batch.comment_text[0].size())

### Integrate Batch Loader into Model

In [0]:
class LSTMModel(nn.Module):
    """
    Class for LSTM recurrent neural network models
    """
    def __init__(self, vocab_size, hidden_dim, pad_idx, batch_size=1,
                 embed_dim=6, weight_decay=0, optimizer_fcn='Adam',
                 learning_rate=1e-3, num_layers=2, dropout=0.05, 
                 num_classes=1, bidirectional=True):
        """
        Create a model based on given paramters.
        
        Arguments:
            vocab_size: the vocabulary size of the model
            hidden_dim: the number of hidden dimensions in the model
            pad_idx: the word-to-index value according to the model instance 
                corpus for padding values
            batch_size: the size of batches the model should expect. 
                Default is 1.
            embed_dim: the number of embedding dimensions. Default is 6. 
            weight_decay: the amount of weight decay or regularization the 
                model optimizer should use. Default is 0.
            optimizer_fcn: the model opimizer to employ. Default is 'Adam.'
            learning_rate: the model optimizer learning rate. Default 1e-3. 
            num_layers: The number of layers the model should deploy. Default 
                is 2.
            dropout: the amount of dropout a model instance should apply. 
                Default is 0.05. 
            num_classes: The number of classes to predict. Default is 1. 
            bidirectional: Whether the model should be bidirectional. Default 
                is True.
        """
        super(LSTMModel, self).__init__()
        nn.Module.__init__(self)
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.embed_dim = embed_dim
        self.dropout = dropout
        self.output_dim = num_classes
        self.loss_fcn = nn.NLLLoss()
        self.weight_decay = weight_decay
        self.learning_rate = learning_rate
        self.pad_idx = pad_idx

        # Layer 1: Embedding Layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_dim, padding_idx=pad_idx)

        # Layer 2: LSTM Layer
        self.lstm = nn.LSTM(input_size = self.embed_dim, 
                            hidden_size = self.hidden_dim,
                            num_layers = self.num_layers, 
                            dropout = self.dropout,
                            bidirectional=bidirectional) 
#                             batch_first=True)

        # Layer 3 (Output Layer): Linear
        self.linear = nn.Linear(self.hidden_dim * 2, self.output_dim)
    
        # Dropout Layer  
        self.dropout_layer = nn.Dropout(self.dropout)

        # define optimizer
        if optimizer_fcn == 'Adam':
            self.optimizer = optim.Adam(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
        elif optimizer_fcn == 'RMSprop':
            self.optimizer = optim.RMSprop(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
        elif optimizer_fcn == 'SDG':
            self.optimizer = optim.SGD(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
            

    def forward(self, input_seq, input_len):
        """
        Pass a single comments' input sequence (vector represntation) through 
        model layers, using the input's length to inform sequence packing for
        batch handling.
        
        H/T to Ben Trevett for example of applying dropout layer and batch packing: 
        https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/
        2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
        
        Arguments:
            input_seq: sentence reqpresentation (word-to-idx vector)
            input_len: the length of the sentence reprensentation
        """
        # input_seq shape is [batch_max_input_len, batch_size]
        embed_out = self.embedding(input_seq)
        
        embed_out = self.dropout_layer(embed_out)
        # embed_out shape is [batch_max_input_len, batch_size, embed_dim]

        # tell LSTM layer to focus on non-padded elements by packing embeddings
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embed_out, input_len)
        
        packed_lstm_out, (hn, cn) = self.lstm(packed_embedded)
        # hn shape is [num_layers * num_directions, batch_size, hidden_dim]
        
        #concat final forward and backward hidden layers to apply any dropout
        hn = self.dropout_layer(torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1))
        # hn shape is [batch size, hid dim * num directions]
        
        out = F.log_softmax(self.linear(hn), dim=1)
        # out shape is [batch_size, outpu_dim]
        
        return out


In [0]:
def train_clf(model, iterator):
    """
    Iterate through batches of input sequence (vector representations) for a 
    single epoch, passing them through model layers, calculating loss, 
    performing a backward pass, stepping the optimzer, and then zero-ing out 
    gradient for the next batch. 
    
    Arguments:
        model: a LSTMModel instance
        iterator: a TorchText data loader
    """
    running_loss = 0
    epoch_recall = 0
    epoch_nontarget_recall = 0
    
    model.train()

    for batch in iterator:
        if len(batch) != iterator.batch_size:
            print(f"Skipping small batch of length {len(batch)}...")
            continue
          
        model.optimizer.zero_grad()

        seq, seq_lengths = batch.comment_text
        y = batch.label.long()

        if torch.cuda.is_available():
            seq = seq.cuda() 
            y = y.cuda()
            seq_lengths = seq_lengths.cuda()

        preds = model.forward(seq, seq_lengths)

        loss = model.loss_fcn(input=preds.squeeze(1), 
                                     target=y)

        y_true = [single_tensor.item() for single_tensor in y]

        indices = torch.argmax(preds, 1)
        y_pred = [single_tensor.item() for single_tensor in indices.long()]

        recall_toxic = recall_score(y_true, y_pred, pos_label=1)
        recall_nontoxic = recall_score(y_true, y_pred, pos_label=0)

        loss.backward()

        running_loss += loss.item()
        epoch_recall += recall_toxic.item()
        epoch_nontarget_recall += recall_nontoxic.item()

        model.optimizer.step()
        
    last_mean_error = np.mean(np.abs(loss.item()))
    return running_loss / len(iterator), epoch_recall / len(iterator),  epoch_nontarget_recall / len(iterator), last_mean_error


In [0]:
def evaluate_clf(model, iterator):
    """
    Evaluate a trained model on a previously unseen data set in terms of 
    F1, Precision, Recall, Accuracy, and AUC-ROC metrics.
    
    Arguments:
        model: a LSTMModel instance
        iterator: a TorchText data loader
    """
    running_loss = 0
    epoch_recall = 0
    epoch_nontarget_recall = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            seq, seq_lengths = batch.comment_text
            
            y = batch.label.long()
            
            # get predictions from model
            predictions = model.forward(seq, seq_lengths)
            
            # calculate loss (solely for comparison with training loss)
            loss = model.loss_fcn(predictions.squeeze(1), y)
            
            # create numerical representation of GPU object, which is more 
            # compatible with numpy score functions
            val_true = [single_tensor.item() for single_tensor in y]

            # take larger of two indices to determine whether prediction is 1
            # ("toxic") or 0 ("not toxic")
            indices = torch.argmax(predictions, 1)
            
            # create numerical representation of GPU object, which is more 
            # compatible with numpy score functions
            val_pred = [single_tensor.item() for single_tensor in indices]

            # calculate metrics for target and non-target groups
            f1_toxic = f1_score(val_true, val_pred, pos_label=1)
            f1_nontoxic = f1_score(val_true, val_pred, pos_label=0)
        
            recall_toxic = recall_score(val_true, val_pred, pos_label=1)
            recall_nontoxic = recall_score(val_true, val_pred, pos_label=0)
            
            precision_toxic = precision_score(val_true, val_pred, pos_label=1)
            precision_nontoxic = precision_score(val_true, val_pred, pos_label=0)
            
            # calculate overall metrics: accuracy, auc_roc
            accuracy = accuracy_score(val_true, val_pred)
            
            try:
                auc_roc = roc_auc_score(val_true, val_pred)
            except:
                auc_roc = -1

            # increment running loss
            running_loss += loss.item()
            
            # increment recall contribution for eventual epoch average 
            epoch_recall += recall_toxic.item()
            epoch_nontarget_recall += recall_nontoxic.item()
  
            metrics = {
                "target": {"f1": round(f1_toxic * 100, 5),
                           "precision": round(precision_toxic * 100, 5),
                           "recall": round(recall_toxic * 100, 5)
                          },
                "nontarget": {"f1": round(f1_nontoxic * 100, 5),
                           "precision": round(precision_nontoxic * 100, 5),
                           "recall": round(recall_nontoxic * 100)
                          },
                "overall": {"accuracy": round(accuracy * 100, 5),
                           "auc_roc": round(auc_roc * 100, 5)}
            }
   
        
    return running_loss / len(iterator), epoch_recall / len(iterator),  epoch_nontarget_recall / len(iterator), metrics

In [0]:
import time

def epoch_time(start_time, end_time):
    """
    Record the amount of time required for each epoch.
    """
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
def loop_epochs(model, train_iter, val_iter, num_epochs, outfile=None, 
                model_prefix=None):  
    """
    Perform model training and evaluation over a given number of epochs for a 
    given model and iterators based on the model, optionally recording results 
    into a given outfile.
    
    Arguments:
        model: an LSTMModel intstance 
        train_iter: a data loader for training with an LSTMModel instance
        val_iter: a data loader for evaluation on an LSTMModel's performance
        num_epochs: the number of epochs for which an LSTMModel should be 
            trained
        outfile: (optional) a string indicating the name of a file in which 
            to record model results, or None. Default is None.
        model_prefix: (optional) a prefix to add to names of model save state 
            files when best epoch model states are stored.
    """
    loop_start = time.time()
    if outfile:
        f = open(outfile, "a")
    else:
        f = None
      
    # best_valid_loss = float('inf')
    last_mean_error = -math.inf
    best_target_recall = -1
    best_nontarget_recall = -1

    thresholds= {
            5000: 1000,
            1000: 100,
            100: 10,
            20: 5,
            10: 3
        }
    metric_results = dict()

    for epoch in range(num_epochs):

        # Record epoch start time
        start_time = time.time()

        print(f"Epoch {epoch}:\nStarting training...", file=f)
        train_loss, target_recall, nontarget_recall, epoch_mean_error = train_clf(model, train_iter)

        end_train = time.time()
        train_mins, train_secs = epoch_time(start_time, end_train)

        print(f"Starting evaluation...({train_mins:02}:{train_secs:02} elapsed)", file=f)
        valid_loss, val_target_recall, val_nontarget_recall, metrics = evaluate_clf(model, val_iter)

        end_eval = time.time()
        eval_mins, eval_secs = epoch_time(end_train, end_eval)
        print(f"Finished evaluation...({eval_mins:02}:{eval_secs:02} elapsed)", file=f)

        epoch_mins, epoch_secs = epoch_time(start_time, end_eval)

        metric_results[epoch] = metrics

        # Compare and record bast recall for "toxic" class
        if val_target_recall > best_target_recall:
            best_target_recall = val_target_recall
            savestate_file = f'{model_prefix}_epoch{epoch}_target.pt'
            torch.save(model.state_dict(), savestate_file)
          
        # Compare and record bast recall for "not toxic" class
        if val_nontarget_recall > best_nontarget_recall:
            best_nontarget_recall = val_nontarget_recall
            savestate_file = f'{model_prefix}_epoch{epoch}_nontarget.pt'
            torch.save(model.state_dict(), savestate_file)     

        # Print or record epoch results and timing
        print(f'Epoch: {epoch+1:02} | Total Epoch Time: {epoch_mins}m {epoch_secs}s', file=f)
        print(f'\tNegative Log Linear Loss:  Loss: {train_loss:.6f} | Target Recall: {target_recall*100:.2f}% | Nontarget Recall: {nontarget_recall*100:.2f}%', file=f)
        print(f'\tVal. Loss: {valid_loss:.6f} | Target Recall: {val_target_recall*100:.2f}% | Nontarget Recall: {val_nontarget_recall*100:.2f}%', file=f)
        
    # Print or record total training and evaluation time 
    loop_mins, loop_secs = epoch_time(loop_start, end_eval)
    print(f"Total Time: {loop_mins:02}:{loop_secs:02} elapsed", file=f)
    print('', file=f)
    print(metric_results, file=f)
    print('', file=f)
    print('', file=f)
    f.close()
    
    if outfile:
        send_to_bucket(filename=outfile, directory="model_results")
    
        send_to_bucket(filename=savestate_file, directory="model_states")


### Test Model Functions on 20% of Dataset

In [0]:
NUM_EPOCHS = 6
hist_lstm = np.zeros(NUM_EPOCHS)
model_name="20_pct_dataset"

In [0]:
lstm_model = LSTMModel(vocab_size = VOCAB_SIZE, hidden_dim=100, num_layers=1, 
#                        embed_dim=300, 
                       embed_dim=200, 
                       batch_size=train_iter.batch_size,
                       dropout=0, num_classes=2, 
                       pad_idx=TEXT.vocab.stoi[TEXT.pad_token])

In [0]:
torch.cuda.is_available()

In [0]:
if torch.cuda.is_available():
    lstm_model = lstm_model.cuda()
    lstm_model.loss_fcn = lstm_model.loss_fcn.cuda()

In [0]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

In [0]:
lstm_model.embedding.weight.data.copy_(pretrained_embeddings)

In [0]:
# let model learn from unknown token, while leaving padding as zeros
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

lstm_model.embedding.weight.data[UNK_IDX] = torch.zeros(lstm_model.embed_dim)
lstm_model.embedding.weight.data[lstm_model.pad_idx] = torch.zeros(lstm_model.embed_dim)

print(lstm_model.embedding.weight.data)

In [0]:
loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
            outfile='20pct_imbalanced.txt', model_prefix='20pct_imbalanced')

## Modeling on Rebalanced Datasets

In [0]:
BATCH_SIZE = 500

In [0]:
def initiate_model(train_df, val_df, batch_size=BATCH_SIZE):
    """
    Perform steps to create a model and related iterators based on given train 
    and test dataframe, and a given batch size.
    
    Arguments:
        train_df dataframe containing training data
        val_df: dataframe containing test data
        batch_size: batch size for data loaders and model dimensions
    """
    
    TEXT = Field(tokenize=tokenizer, lower=True, include_lengths = True)
    LABEL = Field(sequential=False, unk_token=None, 
              dtype=torch.float)
    
    train_ds = DataFrameDataset(train_df, fields = {'label' : LABEL, 
                                                        'comment_text': TEXT})
    valid_ds = DataFrameDataset(val_df, fields = {'label' : LABEL, 
                                                       'comment_text': TEXT})

    print("Torchtext Datasets created...")
    TEXT.build_vocab(train_ds, max_size=25000, 
                   # optional for larger vector size
                   # vectors = "glove.840B.300d",
                 vectors = "glove.6B.200d",
                 unk_init = torch.Tensor.normal_)
    LABEL.build_vocab(train_ds)
    
    print("Torchtext vocabulary cerated...")
    
    VOCAB_SIZE = len(TEXT.vocab)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

    train_iter, val_iter = BucketIterator.splits(
    (train_ds, valid_ds), 
    batch_size = batch_size,
    sort_key=lambda x: len(x.comment_text),
    sort_within_batch = True,
    device = device)
    
    print("Torchtext iterators cerated...")
    
    lstm_model = LSTMModel(vocab_size = VOCAB_SIZE, hidden_dim=100, num_layers=1, 
                           embed_dim=200, 
                           batch_size=train_iter.batch_size,
                           dropout=0, num_classes=2, 
                           pad_idx=TEXT.vocab.stoi[TEXT.pad_token])
    
    print("Model created...")
    
    if torch.cuda.is_available():
        lstm_model = lstm_model.cuda()
        lstm_model.loss_fcn = lstm_model.loss_fcn.cuda()
        
    pretrained_embeddings = TEXT.vocab.vectors
    lstm_model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    lstm_model.embedding.weight.data[UNK_IDX] = torch.zeros(lstm_model.embed_dim)
    lstm_model.embedding.weight.data[lstm_model.pad_idx] = torch.zeros(lstm_model.embed_dim)
    print("Pre-trained embeddings applied...")
    
    return lstm_model, train_iter, val_iter


In [0]:
rebalance_dict = {0: 35, 1: 50, 2: 60, 3: 65, 4:75, 5: 'random'}
data_proportions = [0.2, 0.4, 0.6, 0.8]

### Rebalanced Toxicity on 20% of Dataset

In [0]:

prepared_35, prepared_50, prepared_60, prepared_65, prepared_75, random_df = exl.rebalance_data(train_set)

In [0]:
test_ratio=0.2

for i, df in enumerate([prepared_35, prepared_50, prepared_60, prepared_65, prepared_75, random_df]):
    model_name=f'20pct_model_{rebalance_dict[i]}toxic'
    print(f'Model: {model_name}')
    val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), random_state=1008 )
    
    print(f"Validation sample_created...")
    
    lstm_model, train_iter, val_iter = initiate_model(df, val_sample_df, batch_size=BATCH_SIZE)
    
    print(f"Beginning training and evaluation loops...")

    loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
            outfile=f'{model_name}.txt', model_prefix=model_name)


### Rebalanced Toxicity on 40%, 60%, 80% of Dataset

In [0]:
NUM_EPOCHS = 6

### Training and Evaluation Loop

Run training and evaluation on all proportions of data and toxicity ratios

In [0]:
for proportion in data_proportions[3:]:
    train_set, val_set, test_set = exl.get_samples(train_full, proportion=proportion, 
                                                  train_test_ratio=(1-test_ratio))
    
    pickle_to_gcs(test_set, f'{proportion*100:.0f}pct_test_set.pkl', 
                  directory="test_sets")
        
    rebalanced_dfs = exl.rebalance_data(train_set)
    
    
    
    for i, df in enumerate(rebalanced_dfs):
        if (rebalance_dict[i] in {35}) and (proportion == 0.8):
            continue
   
        model_name=f'{proportion*100:.0f}pct_model_{rebalance_dict[i]}toxic'
        
        print(f'Model: {model_name}')
        
        pickle_to_gcs(df, f'{model_name}_train.pkl', directory="train_sets")
        
        try:
            val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), 
                                           random_state=1008 )
        except:
            val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), 
                                           random_state=1008, replace = True )
            
        pickle_to_gcs(val_sample_df, f'{model_name}_val.pkl', directory="val_sets")

        print(f"Validation sample_created...")

        lstm_model, train_iter, val_iter = initiate_model(df, val_sample_df, 
                                                          batch_size=BATCH_SIZE)

        print(f"Beginning training and evaluation loops...")

        loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
                outfile=f'{model_name}.txt', model_prefix=model_name)


Retrieving training samples at proportion 0.8
Rebalance Ratio: 0.35, 404283 toxic samples out of 1155095
Rebalance Ratio: 0.5, 577547 toxic samples out of 1155095
Rebalance Ratio: 0.6, 693057 toxic samples out of 1155095
Rebalance Ratio: 0.65, 750811 toxic samples out of 1155095
Rebalance Ratio: 0.75, 866321 toxic samples out of 1155095
Rebalanced dfs created...
Model: 80pct_model_35toxic
Validation sample_created...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/80pct_model_35toxic.txt [Content-Type=text/plain]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      
Copying file:///content/80pct_model_35toxic_epoch5_nontarget.pt [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.0 MiB

### Large Dataset Troubleshooting
As 80% datasets are crashing due to too much RAM usage, create all dataframes,  pickle, and run individually.

In [0]:

train_set, val_set, test_set = exl.get_samples(train_full, proportion=0.8, 
                                              train_test_ratio=(1-test_ratio))

pickle_to_gcs(test_set, f'80pct_test_set_past35.pkl', 
              directory="test_sets")

rebalanced_dfs = exl.rebalance_data(train_set)

for i, df in enumerate(rebalanced_dfs):
    if i == 0:
        continue
    model_name = f'80pct_model_{rebalance_dict[i]}toxic'
    pickle_to_gcs(df, f'{model_name}_train.pkl', directory="train_sets")
    
    try:
        val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), 
                                           random_state=1008 )
        pickle_to_gcs(val_sample_df, f'{model_name}_val.pkl', directory="val_sets")
        
    except:
        val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), 
                                       random_state=1008, replace = True )
        
        pickle_to_gcs(val_sample_df, f'{model_name}_val.pkl', directory="val_sets")

Retrieving training samples at proportion 0.8


/usr/local/lib/python3.6/dist-packages/gcsfs/core.py:304: ResourceWarning: unclosed <ssl.SSLSocket fd=85, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 57806), raddr=('74.125.142.95', 443)>
  self._singleton[0] = self
/usr/local/lib/python3.6/dist-packages/gcsfs/core.py:304: ResourceWarning: unclosed <ssl.SSLSocket fd=84, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 34352), raddr=('74.125.20.84', 443)>
  self._singleton[0] = self


Rebalance Ratio: 0.35, 404273 toxic samples out of 1155068
Rebalance Ratio: 0.5, 577534 toxic samples out of 1155068
Rebalance Ratio: 0.6, 693040 toxic samples out of 1155068
Rebalance Ratio: 0.65, 750794 toxic samples out of 1155068
Rebalance Ratio: 0.75, 866301 toxic samples out of 1155068
Rebalanced dfs created...


In [0]:
def run_given_dfs(train_pkl, val_pkl, gcs=False):
    """
    Run all modeling steps from iterator and model creation through training 
    and evaluation of model on existing dataframes stored as pickle files, 
    optionally performing extra steps if pickle files stored in Google Cloud 
    Storage.
    
    Arguments:
        train_pkl: Name of .pkl file of training data frame.
        val_pkl: Name of .pkl file of test data frame.
        gcs: Whether files are stored in Google Cloud Storage. Default is 
            False.
    """
    if gcs:   
        model_name = train_pkl.replace('_train.pkl', '')
        train_df = load_pickle_from_gcs(train_pkl, directory="train_sets") 
        val_df = load_pickle_from_gcs(val_pkl, directory="val_sets")  
    
    else:
        model_name = train_pkl.replace('_train.pkl', '')
        train_df = pd.read_pickle(train_pkl)
        val_df = pd.read_pickle(val_pkl)
    
    print("Initiating model and iterators...")

    lstm_model, train_iter, val_iter = initiate_model(train_df, val_df, 
                                                      batch_size=BATCH_SIZE)

    print(f"Beginning training and evaluation loops...")

    loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
            outfile=f'{model_name}.txt', model_prefix=model_name)
    
    print("Done!")


In [0]:
run_given_dfs('80pct_model_50toxic_train.pkl', '80pct_model_50toxic_val.pkl')

Initiated model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Copying file:///content/80pct_model_50toxic.txt [Content-Type=text/plain]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      
Copying file:///content/80pct_model_50toxic_epoch4_nontarget.pt [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.0 MiB.                                     


In [0]:
run_given_dfs('40pct_model_35toxic_train.pkl', '40pct_model_35toxic_val.pkl', gcs=True)


Initiating model and iterators...
Torchtext Datasets created...


.vector_cache/glove.6B.zip: 862MB [00:32, 26.2MB/s]                           
100%|█████████▉| 398557/400000 [00:27<00:00, 15233.09it/s]

Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...


100%|█████████▉| 398557/400000 [00:40<00:00, 15233.09it/s]

Copying file:///content/40pct_model_35toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/40pct_model_35toxic_epoch4_nontarget.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('40pct_model_65toxic_train.pkl', '40pct_model_65toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/40pct_model_65toxic.txt [Content-Type=text/plain]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      
Copying file:///content/40pct_model_65toxic_epoch4_target.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('40pct_model_75toxic_train.pkl', '40pct_model_75toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/40pct_model_75toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/40pct_model_75toxic_epoch3_target.pt [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('60pct_model_35toxic_train.pkl', '60pct_model_35toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/60pct_model_35toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/60pct_model_35toxic_epoch5_nontarget.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('80pct_model_60toxic_train.pkl', '80pct_model_60toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/80pct_model_60toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/80pct_model_60toxic_epoch4_target.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('80pct_model_65toxic_train.pkl', '80pct_model_65toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/80pct_model_65toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/80pct_model_65toxic_epoch4_target.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('80pct_model_75toxic_train.pkl', '80pct_model_75toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/80pct_model_75toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/80pct_model_75toxic_epoch5_target.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('80pct_model_randomtoxic_train.pkl', '80pct_model_randomtoxic_val.pkl', gcs=True)

/usr/local/lib/python3.6/dist-packages/gcsfs/core.py:304: ResourceWarning: unclosed <ssl.SSLSocket fd=67, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 43386), raddr=('216.58.197.170', 443)>
  self._singleton[0] = self
/usr/local/lib/python3.6/dist-packages/gcsfs/core.py:304: ResourceWarning: unclosed <ssl.SSLSocket fd=66, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 47590), raddr=('172.217.31.173', 443)>
  self._singleton[0] = self


Initiating model and iterators...
Torchtext Datasets created...


.vector_cache/glove.6B.zip: 862MB [16:46, 857kB/s]                            
100%|█████████▉| 398622/400000 [00:27<00:00, 14588.83it/s]

Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...


100%|█████████▉| 398622/400000 [00:40<00:00, 14588.83it/s]

Pre-trained embeddings applied...
Beginning training and evaluation loops...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Copying file:///content/80pct_model_randomtoxic.txt [Content-Type=text/plain]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      
Copying file:///content/80pct_model_randomtoxic_epoch3_target.pt [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.0 MiB.                                     
Done!


## Compare Results

In [0]:
import json

In [0]:
with open('lstm_results.json', 'r') as f:
    results_dict = json.load(f)

In [0]:
reorg_results = {(k1, k2, k3):v3 for k1,v1 in results_dict.items() \
                       for k2,v2 in results_dict[k1].items()
                       for k3,v3 in results_dict[k1][k2].items()}

In [0]:
results_df = pd.DataFrame([reorg_results[i] for i in sorted(reorg_results)],
                  index=pd.MultiIndex.from_tuples([i for i in sorted(reorg_results.keys())]))    

In [0]:
results_df = results_df.unstack(level=2)

In [0]:
results_df.style

### Selecting a Model

80% of Data, 65% toxic comments, vs. 20% of Data, 75% toxic, vs. 40% of Data, 60% toxic.

Selected model: **40% of data, 60% toxic**

In [0]:
results_df.loc['40pct_model_60toxic']   

accuracy                    auc_roc  ... precision    recall                  
  nontarget   overall target nontarget  ...    target nontarget overall    target
0       NaN  88.16372    NaN       NaN  ...  99.48254      88.0     NaN  88.18807
1       NaN  93.14159    NaN       NaN  ...  98.91304      72.0     NaN  93.92202
2       NaN  91.92478    NaN       NaN  ...  99.01840      75.0     NaN  92.54587
3       NaN  92.80973    NaN       NaN  ...  98.55596      62.0     NaN  93.92202
4       NaN  93.58407    NaN       NaN  ...  98.22275      53.0     NaN  95.06881
5       NaN  94.13717    NaN       NaN  ...  98.23322      53.0     NaN  95.64220

[6 rows x 15 columns]

In [0]:
results_df.loc['model_results_80pct_model_65toxic'] 

accuracy                    auc_roc  ... precision    recall                  
  nontarget   overall target nontarget  ...    target nontarget overall    target
0       NaN  93.07692    NaN       NaN  ...  96.63866      60.0     NaN  95.83333
1       NaN  95.38462    NaN       NaN  ...  96.72131      60.0     NaN  98.33333
2       NaN  95.38462    NaN       NaN  ...  96.72131      60.0     NaN  98.33333
3       NaN  93.84615    NaN       NaN  ...  96.66667      60.0     NaN  96.66667
4       NaN  93.84615    NaN       NaN  ...  96.66667      60.0     NaN  96.66667
5       NaN  95.38462    NaN       NaN  ...  96.72131      60.0     NaN  98.33333

[6 rows x 15 columns]

In [0]:
results_df.loc['20pct_model_75toxic']

accuracy                    auc_roc  ... precision    recall                  
  nontarget   overall target nontarget  ...    target nontarget overall    target
0       NaN  86.78279    NaN       NaN  ...  99.39099      86.0     NaN  86.80851
1       NaN  87.29508    NaN       NaN  ...  99.39467      86.0     NaN  87.34043
2       NaN  92.52049    NaN       NaN  ...  98.00664      50.0     NaN  94.14894
3       NaN  91.70082    NaN       NaN  ...  98.09630      53.0     NaN  93.19149
4       NaN  93.64754    NaN       NaN  ...  97.92576      47.0     NaN  95.42553
5       NaN  92.21311    NaN       NaN  ...  98.43049      61.0     NaN  93.40426

[6 rows x 15 columns]

In [0]:
results_df.loc['model_results_60pct_model_75toxic']

accuracy                     auc_roc  ...  precision    recall               
  nontarget    overall target nontarget  ...     target nontarget overall target
0       NaN  100.00000    NaN       NaN  ...  100.00000     100.0     NaN  100.0
1       NaN   90.90909    NaN       NaN  ...   90.90909       0.0     NaN  100.0
2       NaN   90.90909    NaN       NaN  ...   90.90909       0.0     NaN  100.0
3       NaN   90.90909    NaN       NaN  ...   90.90909       0.0     NaN  100.0
4       NaN   90.90909    NaN       NaN  ...   90.90909       0.0     NaN  100.0
5       NaN   90.90909    NaN       NaN  ...   90.90909       0.0     NaN  100.0

[6 rows x 15 columns]

In [0]:
results_df.loc['20pct_model_50toxic']   

accuracy                    auc_roc  ... precision    recall                  
  nontarget   overall target nontarget  ...    target nontarget overall    target
0       NaN  92.69521    NaN       NaN  ...  23.52941      93.0     NaN  72.72727
1       NaN  95.71788    NaN       NaN  ...  36.36364      96.0     NaN  72.72727
2       NaN  95.21411    NaN       NaN  ...  33.33333      96.0     NaN  72.72727
3       NaN  96.22166    NaN       NaN  ...  37.50000      97.0     NaN  54.54545
4       NaN  96.22166    NaN       NaN  ...  35.71429      98.0     NaN  45.45455
5       NaN  95.96977    NaN       NaN  ...  30.76923      98.0     NaN  36.36364

[6 rows x 15 columns]

In [0]:
results_df.loc['model_results_80pct_model_35toxic']

accuracy                     auc_roc  ... precision    recall               
  nontarget    overall target nontarget  ...    target nontarget overall target
0       NaN   91.42857    NaN       NaN  ...      40.0      91.0     NaN  100.0
1       NaN   94.28571    NaN       NaN  ...      50.0      94.0     NaN  100.0
2       NaN  100.00000    NaN       NaN  ...     100.0     100.0     NaN  100.0
3       NaN  100.00000    NaN       NaN  ...     100.0     100.0     NaN  100.0
4       NaN  100.00000    NaN       NaN  ...     100.0     100.0     NaN  100.0
5       NaN   97.14286    NaN       NaN  ...     100.0     100.0     NaN   50.0

[6 rows x 15 columns]

In [0]:
results_df.loc['model_results_60pct_model_60toxic']

accuracy                    auc_roc  ... precision    recall                  
  nontarget   overall target nontarget  ...    target nontarget overall    target
0       NaN  93.42857    NaN       NaN  ...  98.13665      65.0     NaN  94.89489
1       NaN  95.42857    NaN       NaN  ...  97.31343      47.0     NaN  97.89790
2       NaN  95.14286    NaN       NaN  ...  97.30539      47.0     NaN  97.59760
3       NaN  94.57143    NaN       NaN  ...  96.72619      35.0     NaN  97.59760
4       NaN  94.57143    NaN       NaN  ...  96.72619      35.0     NaN  97.59760
5       NaN  94.85714    NaN       NaN  ...  96.46018      29.0     NaN  98.19820

[6 rows x 15 columns]

In [0]:
results_df.loc['model_results_20pct_model_35toxic']

accuracy                    auc_roc  ... precision    recall               
  nontarget   overall target nontarget  ...    target nontarget overall target
0       NaN  93.05556    NaN       NaN  ...       0.0      93.0     NaN    0.0
1       NaN  97.22222    NaN       NaN  ...       0.0      97.0     NaN    0.0
2       NaN  98.61111    NaN       NaN  ...       0.0      99.0     NaN    0.0
3       NaN  98.61111    NaN       NaN  ...       0.0      99.0     NaN    0.0
4       NaN  98.61111    NaN       NaN  ...       0.0      99.0     NaN    0.0
5       NaN  97.22222    NaN       NaN  ...       0.0      97.0     NaN    0.0

[6 rows x 15 columns]

In [0]:
results_df.loc['model_results_80pct_model_75toxic']

accuracy                    auc_roc  ... precision    recall                   
  nontarget   overall target nontarget  ...    target nontarget overall     target
0       NaN  94.94949    NaN       NaN  ...  96.84211      40.0     NaN   97.87234
1       NaN  93.93939    NaN       NaN  ...  96.80851      40.0     NaN   96.80851
2       NaN  96.96970    NaN       NaN  ...  96.90722      40.0     NaN  100.00000
3       NaN  93.93939    NaN       NaN  ...  96.80851      40.0     NaN   96.80851
4       NaN  94.94949    NaN       NaN  ...  96.84211      40.0     NaN   97.87234
5       NaN  95.95960    NaN       NaN  ...  96.87500      40.0     NaN   98.93617

[6 rows x 15 columns]

In [0]:
results_df.loc['model_results_20pct_model_60toxic']

accuracy                    auc_roc  ...  precision    recall                  
  nontarget   overall target nontarget  ...     target nontarget overall    target
0       NaN  91.11111    NaN       NaN  ...  100.00000     100.0     NaN  90.96045
1       NaN  95.00000    NaN       NaN  ...   99.41176      67.0     NaN  95.48023
2       NaN  95.55556    NaN       NaN  ...   99.41520      67.0     NaN  96.04520
3       NaN  96.66667    NaN       NaN  ...   98.85714      33.0     NaN  97.74011
4       NaN  97.77778    NaN       NaN  ...   99.42857      67.0     NaN  98.30508
5       NaN  97.77778    NaN       NaN  ...   98.87006      33.0     NaN  98.87006

[6 rows x 15 columns]